In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
airpass = pd.read_csv('C:/Users/U_M1P8G/Desktop/IDE/Доп. главы высшей математики/Ансамблевые методы/data/AirPass.csv')

In [27]:
airpass = airpass.drop(airpass.columns[0], axis=1)
airpass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 103904 non-null  int64  
 1   Gender                             103904 non-null  object 
 2   Customer Type                      103904 non-null  object 
 3   Age                                103904 non-null  int64  
 4   Type of Travel                     103904 non-null  object 
 5   Class                              103904 non-null  object 
 6   Flight Distance                    103904 non-null  int64  
 7   Inflight wifi service              103904 non-null  int64  
 8   Departure/Arrival time convenient  103904 non-null  int64  
 9   Ease of Online booking             103904 non-null  int64  
 10  Gate location                      103904 non-null  int64  
 11  Food and drink                     1039

In [28]:
airpass['Arrival Delay in Minutes'] = airpass['Arrival Delay in Minutes'].fillna(
    airpass['Arrival Delay in Minutes'].median()
)

In [29]:
airpass['Arrival Delay in Minutes'].mean()

np.float64(15.133392362180475)

In [30]:
pivot_air = airpass.groupby('Class')['satisfaction'].value_counts(normalize=True)
pivot_air

Class     satisfaction           
Business  satisfied                  0.694251
          neutral or dissatisfied    0.305749
Eco       neutral or dissatisfied    0.813862
          satisfied                  0.186138
Eco Plus  neutral or dissatisfied    0.753936
          satisfied                  0.246064
Name: proportion, dtype: float64

Перекодируем часть бинарных признаков, чтобы использовать их при обучении:

In [31]:
airpass['satisfaction'] = airpass['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
airpass['Customer Type'] = airpass['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
airpass['Type of Travel'] = airpass['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})
airpass['Gender'] = airpass['Gender'].map({'Male': 0, 'Female': 1})

In [32]:
# Создаем dummy-переменные
airpass = pd.get_dummies(airpass)

In [33]:
airpass.shape

(103904, 26)

In [35]:
# Разбиваем на обучающую и тестовую выборки 80/20
X = airpass.drop('satisfaction', axis=1)
y = airpass['satisfaction']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=26
)
X_test.shape[0]


20781

In [37]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # обучаем и преобразуем train
X_test_scaled = scaler.transform(X_test)        # преобразуем test тем же scaler

print(X_test_scaled)

[[ 0.94082514 -1.0182236   0.47381845 ...  1.04624209 -0.90523003
  -0.27897268]
 [-0.86687265  0.98210256  0.47381845 ...  1.04624209 -0.90523003
  -0.27897268]
 [-1.63303947  0.98210256 -2.11051299 ... -0.95580173  1.10469159
  -0.27897268]
 ...
 [ 1.52620083 -1.0182236  -2.11051299 ... -0.95580173  1.10469159
  -0.27897268]
 [-0.60330678 -1.0182236   0.47381845 ... -0.95580173  1.10469159
  -0.27897268]
 [ 1.34108544 -1.0182236   0.47381845 ...  1.04624209 -0.90523003
  -0.27897268]]


Логистическая регрессия

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score


clf = LogisticRegression()
clf.fit(X_train_scaled, y_train)
preds_train = clf.predict(X_train_scaled)
preds_test = clf.predict(X_test_scaled)


In [40]:
f1_train = f1_score(y_train, preds_train)
f1_test = f1_score(y_test, preds_test)

print(f"F1-score на обучающей выборке: {f1_train:.3f}")
print(f"F1-score на тестовой выборке:  {f1_test:.3f}")

F1-score на обучающей выборке: 0.854
F1-score на тестовой выборке:  0.855


AdaBoost    

In [41]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

# Создаем базовое дерево с random_state=26
base_tree = DecisionTreeClassifier(random_state=26)

# Создаем и обучаем AdaBoost
ada_model = AdaBoostClassifier(
    estimator=base_tree,
    learning_rate=0.01,
    random_state=26
)
ada_model.fit(X_train_scaled, y_train)

# Делаем предсказания
y_pred_test = ada_model.predict(X_test_scaled)

# Вычисляем F1-score
f1_ada = f1_score(y_test, y_pred_test)

print(f"F1-score AdaBoost: {f1_ada:.3f}")

F1-score AdaBoost: 0.940


Градиентный бустинг

In [42]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

# Сетка параметров
params = {
    "n_estimators": 2**np.arange(8),  # [1, 2, 4, 8, 16, 32, 64, 128]
    "learning_rate": 0.1**np.arange(3)  # [1.0, 0.1, 0.01]
}

# Создаем модель и GridSearchCV
gb_model = GradientBoostingClassifier(random_state=26)
grid_search = GridSearchCV(
    gb_model,
    params,
    cv=3,  # кросс-валидация на 3 фолда
    scoring='f1',  # используем F1-score как метрику
    n_jobs=-1,  # использовать все ядра процессора
    verbose=1
)

# Запускаем поиск
grid_search.fit(X_train_scaled, y_train)

# Лучшие параметры и лучший score
best_f1 = grid_search.best_score_
best_params = grid_search.best_params_

print("РЕЗУЛЬТАТЫ GRID SEARCH")
print("=" * 40)
print(f"Лучшие параметры: {best_params}")
print(f"Наилучший F1-score: {best_f1:.3f}")

Fitting 3 folds for each of 24 candidates, totalling 72 fits
РЕЗУЛЬТАТЫ GRID SEARCH
Лучшие параметры: {'learning_rate': np.float64(1.0), 'n_estimators': np.int64(128)}
Наилучший F1-score: 0.949


XGBoost

In [43]:
from xgboost import XGBClassifier

# Создаем и обучаем XGBoost с random_state=26
xgb_model = XGBClassifier(random_state=26)
xgb_model.fit(X_train_scaled, y_train)

# Делаем предсказания
y_pred_test = xgb_model.predict(X_test_scaled)

# Вычисляем F1-score
f1_xgb = f1_score(y_test, y_pred_test)

print(f"F1-score XGBoost: {f1_xgb:.3f}")

F1-score XGBoost: 0.957


CatBoost

In [44]:
from catboost import CatBoostClassifier

# Создаем и обучаем XGBoost с random_state=26
cat_model = CatBoostClassifier(random_state=26)
cat_model.fit(X_train_scaled, y_train)

# Делаем предсказания
y_pred_test = cat_model.predict(X_test_scaled)

# Вычисляем F1-score
f1_cat = f1_score(y_test, y_pred_test)

print(f"F1-score CatBoost: {f1_cat:.3f}")

Learning rate set to 0.068023
0:	learn: 0.6018089	total: 182ms	remaining: 3m 1s
1:	learn: 0.5020769	total: 199ms	remaining: 1m 39s
2:	learn: 0.4472481	total: 228ms	remaining: 1m 15s
3:	learn: 0.4028675	total: 254ms	remaining: 1m 3s
4:	learn: 0.3674724	total: 277ms	remaining: 55.1s
5:	learn: 0.3397844	total: 295ms	remaining: 48.9s
6:	learn: 0.3121211	total: 314ms	remaining: 44.5s
7:	learn: 0.2917499	total: 329ms	remaining: 40.8s
8:	learn: 0.2749039	total: 345ms	remaining: 38s
9:	learn: 0.2575191	total: 361ms	remaining: 35.8s
10:	learn: 0.2473690	total: 377ms	remaining: 33.9s
11:	learn: 0.2377531	total: 392ms	remaining: 32.3s
12:	learn: 0.2279309	total: 405ms	remaining: 30.8s
13:	learn: 0.2212512	total: 422ms	remaining: 29.7s
14:	learn: 0.2100359	total: 444ms	remaining: 29.2s
15:	learn: 0.2025733	total: 469ms	remaining: 28.9s
16:	learn: 0.1942303	total: 484ms	remaining: 28s
17:	learn: 0.1877939	total: 499ms	remaining: 27.2s
18:	learn: 0.1832381	total: 514ms	remaining: 26.5s
19:	learn: 0.

Матрица ошибок

In [56]:
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

conf_matrix_test = confusion_matrix(y_test, y_pred_test)


conf_matrix_percent = np.round((conf_matrix_test / len(y_test)) * 100).astype(int)
print(conf_matrix_percent)

[[55  1]
 [ 2 42]]


Важность признаков

In [57]:
feature_names = [x for x in airpass if x != 'satisfaction']
pd.DataFrame({'feat': feature_names,
              'coef': cat_model.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
6,Inflight wifi service,25.364737
4,Type of Travel,18.391876
11,Online boarding,7.401483
2,Customer Type,7.240564
22,Class_Business,5.420057
17,Checkin service,3.925791
3,Age,3.742350
16,Baggage handling,3.640798
9,Gate location,3.196286
12,Seat comfort,3.012455


Стекинг

In [12]:
frogs = pd.read_csv('C:/Users/U_M1P8G/Desktop/IDE/Доп. главы высшей математики/Ансамблевые методы/data/Frogs_MFCCs.csv')

In [13]:
# Разбиваем на обучающую и тестовую выборки 80/20
frogs['Family_binary'] = (frogs['Family'] == 'Dendrobatidae')
X = frogs.drop(['Family', 'Genus', 'Species', 'RecordID'], axis=1)
y = frogs['Family_binary']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=31
)
X_test.shape[0]

1439

Случайный лес

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

model_rfc = RandomForestClassifier(n_estimators=10, random_state=42)
model_rfc.fit(X_train, y_train)
pred = model_rfc.predict(X_test)
f1_score(y_test, pred)

0.9946524064171123

Стекинг

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score

# Базовые модели
base_models = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=31)),
    ('knn', KNeighborsClassifier(n_neighbors=11)),
    ('nb', GaussianNB())
]

# Мета-модель
meta_model = LogisticRegression()

# Стекинг классификатор
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5  # стратифицированная 5-фолд кросс-валидация
)

# Обучение стекинга
stacking_model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred_stacking = stacking_model.predict(X_test)

# F1-score
f1_stacking = f1_score(y_test, y_pred_stacking)

print(f"F1-score стекинга: {f1_stacking:.2f}")

F1-score стекинга: 1.00


In [16]:
frogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7195 entries, 0 to 7194
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MFCCs_ 1       7195 non-null   float64
 1   MFCCs_ 2       7195 non-null   float64
 2   MFCCs_ 3       7195 non-null   float64
 3   MFCCs_ 4       7195 non-null   float64
 4   MFCCs_ 5       7195 non-null   float64
 5   MFCCs_ 6       7195 non-null   float64
 6   MFCCs_ 7       7195 non-null   float64
 7   MFCCs_ 8       7195 non-null   float64
 8   MFCCs_ 9       7195 non-null   float64
 9   MFCCs_10       7195 non-null   float64
 10  MFCCs_11       7195 non-null   float64
 11  MFCCs_12       7195 non-null   float64
 12  MFCCs_13       7195 non-null   float64
 13  MFCCs_14       7195 non-null   float64
 14  MFCCs_15       7195 non-null   float64
 15  MFCCs_16       7195 non-null   float64
 16  MFCCs_17       7195 non-null   float64
 17  MFCCs_18       7195 non-null   float64
 18  MFCCs_19